In [1]:
import pandas as pd
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf

In [2]:
rating_df = pd.read_csv('ratings.csv')
tag_df = pd.read_csv('tags.csv')
movie_df = pd.read_csv('movies.csv')
rating_df['userId'] = rating_df['userId'] - 1
rating_df['movieId'] = rating_df['movieId'] - 1
tag_df['userId'] = tag_df['userId'] - 1
tag_df['movieId'] = tag_df['movieId'] - 1
movie_df['movieId'] = movie_df['movieId'] - 1
num_users = rating_df['userId'].nunique()
num_movies = rating_df['movieId'].nunique()

In [210]:
movie_df['year'] = pd.to_numeric(movie_df['title'].str.strip().str[-5:-1], errors='coerce')
movie_df['year'] =( movie_df['year']-movie_df['year'].mean())/movie_df['year'].std()
movie_df['year'] = movie_df['year'].fillna(0)

In [211]:
ser = movie_df['genres'].str.split('|')


In [212]:
genre_df = movie_df.set_index('movieId')['genres'].str.get_dummies()
genre_df = pd.merge(rating_df[['movieId']], genre_df, on= 'movieId').drop('movieId', axis=1)

In [213]:
rating_df.shape

(100836, 4)

In [216]:
if 'year' not in rating_df.columns:
    rating_df = pd.merge(rating_df, movie_df[['movieId', 'year']], on = 'movieId')

In [245]:
tfph = tf.compat.v1.placeholder
tfvar = tf.compat.v1.get_variable

In [246]:
tf.compat.v1.disable_eager_execution()
tf.compat.v1.reset_default_graph()
user_ph = tfph(shape = (None, ), dtype = tf.int32)
movie_ph = tfph(shape = (None, ), dtype = tf.int32)
emb_size = 32
user_emb = tfvar('users', shape = (num_users,emb_size), dtype = tf.float32)
movie_emb = tfvar('movies', shape = (num_movies,emb_size), dtype = tf.float32)

users = tf.compat.v1.nn.embedding_lookup(user_emb, user_ph)
movies_learned = tf.compat.v1.nn.embedding_lookup(movie_emb, movie_ph)
movie_meta = tf.compat.v1.placeholder(shape = (None, 1 + genre_df.shape[1]), dtype = tf.float32)
movies = tf.concat((movies_learned, movie_meta), axis=1)
# relation_matrix = tf.compat.v1.get_variable('relations', shape = (users.shape[1],movies.shape[1]), dtype = tf.float32)

# preference_alignment = users * movies
# yhat_raw = tf.reduce_sum(preference_alignment, axis=1)
# U * Sigma * Movie
# yhat_raw = tf.reduce_sum(users @ relation_matrix @ movies, axis=1)
# yhat_raw = tf.reduce_sum(tf.reduce_sum(users[:,:,None] * relation_matrix[None, :] * movies[:, None, :], axis=2), axis=1)
features = tf.concat((users, movies), axis=1)
for i in range(3):
    features = tf.compat.v1.layers.Dense(64)(features)
    features = tf.nn.leaky_relu(features)
yhat_raw = tf.compat.v1.layers.Dense(1)(features)[:,0]
yhat = tf.sigmoid(yhat_raw /4) * 4 + 1
rating_ph = tf.compat.v1.placeholder(shape = (None, ), dtype = tf.float32)
loss = tf.reduce_sum(tf.square(yhat - rating_ph))
opt = tf.compat.v1.train.AdamOptimizer().minimize(loss)
sess = tf.compat.v1.Session()
sess.run(tf.compat.v1.global_variables_initializer())

In [247]:
for i in range(1000):
  _, cur_loss = sess.run(
      [opt, loss], {
          user_ph: rating_df['userId'], movie_ph: rating_df['movieId'], rating_ph: rating_df['rating'],
      movie_meta: pd.concat((rating_df[['year']], genre_df), axis=1)})
  if i % 10 == 0:
    print(cur_loss)

133560.44
119054.5
106262.93
98785.47
88468.67
79663.31
74225.516
71242.36
69407.5
68409.11
67711.42
67184.72
66698.61
66178.8
65582.14
64887.234
64093.535
63221.297
62297.742
61331.22
60312.18
59225.54
58042.28
56727.117
55249.72
53580.547
51724.594
49827.207
47853.54
45897.72
44010.758
42514.117
40677.03
39190.26
37781.465
36468.4
35291.062
34146.016
33154.9
32218.164
31364.71
30647.914
29859.824
29150.262
28486.746
27907.568
27446.953
27009.555
26579.434
26095.205
25602.209
25193.967
24950.623
24697.422
24291.045
23913.344
23700.959
23596.451
23209.822
22901.703
22891.92
22625.062
22287.02
22241.582
22078.525
21755.88
21622.047
21575.875
21249.934
21136.932
21092.559
20786.8
20671.594
20710.328
20396.502
20460.312
20287.176
20039.195
20205.566
19895.053
19758.555
19837.967
19536.13
19571.178
19432.158
19355.137
19363.438
19176.71
19102.684
19064.762
18896.057
18925.938
18855.621
18668.406
18748.045
18515.363
18450.424
18400.016
18461.984
18287.525
